In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Cyberlabs- ML/EMNIST dataset/emnist-letters-train.csv')
train.head()

,23,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.406,0.407,0.408,0.409,0.410,0.411,0.412,0.413,0.414,0.415
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test = pd.read_csv('/content/drive/MyDrive/Cyberlabs- ML/EMNIST dataset/emnist-letters-test.csv')
test.head()

,1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.548,0.549,0.550,0.551,0.552,0.553,0.554,0.555,0.556,0.557
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
l_train= train.shape[1]

X_train= train.iloc[:, 1:l_train]
y_train= train.iloc[:, 0]

X_train= X_train.to_numpy()
y_train= y_train.to_numpy()
y_train= y_train.reshape(X_train.shape[0], 1)
print(X_train.shape)
y_train.shape

m= X_train.shape[0]
n= X_train.shape[1]
print(y_train)

X_train= X_train.T
y_train= y_train.T

(88799, 784)
[[ 7]
 [16]
 [15]
 ...
 [ 1]
 [23]
 [12]]


In [6]:
def initial_params():
  W1= np.random.rand(26, n) - 0.5
  b1= np.random.rand(26, 1) - 0.5
  W2= np.random.rand(26, 26) - 0.5
  b2= np.random.rand(26, 1) - 0.5
  return W1, b1, W2, b2

In [7]:
# def sigmoid(Z):
#   return 1/(1+np.exp(-z))

In [8]:
def softMax(z):
  warnings.filterwarnings('ignore')
  z= np.float128(z)
  A= np.exp(z)/ np.sum(np.exp(z))
  return A

In [9]:
def ReLu(z):
  return np.maximum(z, 0)

In [10]:
def ReLu_deriv(z):
  return z>0

In [11]:
def Y_edit(Y):
  Y_edited= np.zeros((Y.shape[0], Y.max() ))
  Y_edited[np.arange(Y.shape[0]), Y-1 ]= 1
  Y_edited= Y_edited.T
  return Y_edited

In [12]:
def forward_prop(W1, b1, W2, b2, X):
  Z1= np.dot(W1, X) + b1
  A1= ReLu(Z1)
  Z2= np.dot(W2, A1) + b2
  A2= softMax(Z2)
  return Z1, A1, Z2, A2

In [13]:
def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
  Y_edited= Y_edit(Y)
  dZ2= A2- Y_edited
  dW2= 1/m * np.dot( dZ2, A1.T )
  db2= 1/m * np.sum(dZ2)
  dZ1= np.dot(W2.T, dZ2) * ReLu_deriv(Z1)
  dW1= 1/m * np.dot( dZ1, X.T)
  db1= 1/m * np.sum(dZ1)
  return dW1, db1, dW2, db2

In [14]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
  W1 = W1 - alpha * dW1
  b1 = b1 - alpha * db1
  W2 = W2 - alpha * dW2
  b2 = b2 - alpha * db2
  return W1, b1, W2, b2

In [15]:
def calc_predictions(A2):
  return np.argmax(A2, 0)

In [16]:
def calc_accuracy(predictions, Y):
  print(predictions, Y)
  return np.sum(predictions == Y)/ Y.shape[0]

In [17]:
def gradientDescent(X, Y, alpha, num_iters):
  W1, b1, W2, b2 = initial_params()

  for i in range(num_iters):
    Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
    dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
    W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

  if i%20 == 0:
    print("Iteration:", i)
    predictions= calc_predictions(A2)
    print(calc_accuracy(predictions, Y))
  return W1, b1, W2, b2

In [ ]:
alpha= 0.1
num_iters= 500
W1, b1, W2, b2 = gradientDescent(X_train, y_train, alpha, num_iters)

In [ ]:
def calculations(X, W1, b1, W2, b2):
  _, _, _, A2= forward_prop(W1, b1, W2, b2, X)
  predictions= calc_predictions(A2)
  return predictions+1

In [ ]:
def test_predictions(index, W1, b1, W2, b2):
  current_image= X_train[:, index, None]
  predictions= calculations(X_train[:, index, None], W1, b1, W2, b2)
  label= y_train[index]
  print("Predictions: ", predictions)
  print("Label: ", label)

In [ ]:
current_image = current_image.reshape((28, 28)) * 255
plt.gray()
plt.imshow(current_image, interpolation='nearest')
plt.show()

In [ ]:
predictions= calculations(X_test, W1, b1, W2, b2)
accuracy= calc_accuracy(predictions, y_test) * 100
print(accuracy)